In [126]:
import pandas as pd
import ast
from datetime import datetime
import numpy as np
from fuzzywuzzy import fuzz
import re, string
from collections import Counter
from sklearn import preprocessing

In [127]:
df_raw = pd.read_csv("./没有sample的最终.csv")

In [3]:
df_raw

,Unnamed: 0,business_id,Cases,Deaths,name,address,city,state,postal_code,latitude,...,categories_Asian Fusion,categories_Mediterranean,total_checkIn,checkIn_2019,chain,covid_cumul_case_state,covid_cumul_death_state,dayOfStart,land_price,status_g
0,0,eBEfgOPG7pvFhb2wcG9I7w,2650.0,222.0,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,...,False,False,1.0,1,True,32033.0,2484.0,522,11225,OPERATIONAL
1,1,lu7vtrp_bE9PnxWfA8g4Pg,2650.0,222.0,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,...,False,False,8.0,0,False,32033.0,2484.0,2020,11225,OPERATIONAL
2,2,9sRGfSVEfLhN_km60YruTA,2650.0,222.0,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,...,False,False,2.0,2,False,32033.0,2484.0,443,11225,CLOSED_TEMPORARILY
3,3,pcaQDBM6r0PWTXfYZK6RdA,26288.0,3068.0,IGA Extra,8130 Boulevard Champlain,Lasalle,QC,H8P 1B4,45.425645,...,False,False,181.0,13,False,52849.0,4978.0,3266,6606,OPERATIONAL
4,4,LoRef3ChgZKbxUio-sHgQg,26288.0,3068.0,Amir,5252 Rue Jean Talon O,MontrÂ©al,QC,H4P 2A7,45.494870,...,False,False,10.0,0,True,52849.0,4978.0,2575,6606,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56169,56169,AlwKuzyrrxlZ04ktebwmcA,824.0,29.0,Pizza Pit,13 Atlas Ct,Madison,WI,53714,43.086793,...,False,False,7.0,0,True,20701.0,646.0,3542,70070,OPERATIONAL
56170,56170,cmsAFAymfVoBitSF8R6GKw,12761.0,489.0,Yao Fine Chinese Cuisine,"2487 S Gilbert Rd, Ste 102",Gilbert,AZ,85295,33.304935,...,False,False,419.0,23,False,25517.0,1047.0,3766,26493,OPERATIONAL
56171,56171,YZeUH6zYS0dq5QHLYZhUnQ,12761.0,489.0,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,...,False,False,253.0,9,True,25517.0,1047.0,3076,26493,OPERATIONAL
56172,56172,hskVqZCPqy-omm9CHi44xQ,5358.0,106.0,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,...,False,False,6.0,0,True,34809.0,1020.0,3396,38976,OPERATIONAL


In [128]:
df_raw = pd.get_dummies(data=df_raw, columns=['status_g'])

In [129]:
list(df_raw.columns)

['Unnamed: 0',
 'business_id',
 'Cases',
 'Deaths',
 'name',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars',
 'review_count',
 'hours',
 'RestaurantsAttire',
 'RestaurantsTakeOut',
 'BusinessAcceptsCreditCards',
 'NoiseLevel',
 'GoodForKids',
 'RestaurantsReservations',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'HasTV',
 'Alcohol',
 'BikeParking',
 'RestaurantsDelivery',
 'ByAppointmentOnly',
 'OutdoorSeating',
 'Caters',
 'WheelchairAccessible',
 'WiFi',
 'RestaurantsTableService',
 'DogsAllowed',
 'HappyHour',
 'parking_garage',
 'parking_street',
 'parking_validated',
 'parking_lot',
 'parking_valet',
 'Ambience_touristy',
 'Ambience_hipster',
 'Ambience_romantic',
 'Ambience_intimate',
 'Ambience_trendy',
 'Ambience_upscale',
 'Ambience_classy',
 'Ambience_casual',
 'Ambience_divey',
 'GoodForMeal_dessert',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'GoodForMeal_dinner',
 'GoodForMeal_brunch',
 'GoodForMeal_breakfast',
 'cate

In [130]:
df_raw.to_csv('./graph_raw.csv', index=False)  

In [131]:
df_raw1 = pd.read_csv("./没有sample的最终.csv")
df_star = df_raw1[['stars', 'status_g']]

In [132]:
set(df_raw1['stars'])

{1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 10.0, 18.0, 1466.0}

In [133]:
star = {1.0: [0,0,0], 1.5: [0,0,0], 2.0: [0,0,0], 2.5: [0,0,0], 3.0: [0,0,0], 3.5: [0,0,0], 4.0: [0,0,0], 4.5: [0,0,0], 5.0: [0,0,0]}
for index, row in df_star.iterrows():
    cur_star = row['stars']
    if cur_star not in star:
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        star[cur_star][0] = star[cur_star][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        star[cur_star][1] = star[cur_star][1] + 1
    elif cur_status == 'OPERATIONAL':
        star[cur_star][2] = star[cur_star][2] + 1
star


{1.0: [56, 15, 393],
 1.5: [79, 48, 1369],
 2.0: [183, 128, 3173],
 2.5: [376, 257, 4951],
 3.0: [480, 488, 8065],
 3.5: [612, 720, 10583],
 4.0: [667, 805, 12096],
 4.5: [550, 409, 7350],
 5.0: [377, 88, 1853]}

In [134]:
df_star = pd.DataFrame.from_dict(star).T

In [135]:
df_star.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_star['star'] = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [136]:
df_star.to_csv('./graph_star.csv', index=False)  

In [137]:
df_star_percentage = pd.DataFrame()
df_star_percentage['star'] = df_star['star']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_star.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum)
df_star_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_star_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_star_percentage['OPERATIONAL'] = OPERATIONAL
df_star_percentage.to_csv('./graph_star_percentage.csv', index=False)  

review

In [120]:
df_raw2 = pd.read_csv("./没有sample的最终.csv")
df_review = df_raw2[['review_count', 'status_g']]
df_review = df_review.dropna()
df_review = df_review[df_review.review_count != "{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22:0', 'Thursday': '17:0-22:0', 'Friday': '17:0-23:0', 'Saturday': '17:0-23:0', 'Sunday': '15:0-21:0'}"]
df_review = df_review[df_review.review_count != "{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Sunday': '11:0-20:0'}"]
df_review = df_review.astype({"review_count": float})
df_review = df_review.astype({"review_count": int})
df_review = df_review[df_review.review_count <= 100]
df_review['review_count'].describe()

count    45816.000000
mean        24.723197
std         24.310530
min          3.000000
25%          6.000000
50%         15.000000
75%         36.000000
max        100.000000
Name: review_count, dtype: float64

In [121]:
review_count = {}
for item in range(100):
    review_count[item] = [0, 0, 0]
for index, row in df_review.iterrows():
    cur_review_count = row['review_count']
    if cur_review_count not in review_count.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        review_count[cur_review_count][0] = review_count[cur_review_count][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        review_count[cur_review_count][1] = review_count[cur_review_count][1] + 1
    elif cur_status == 'OPERATIONAL':
        review_count[cur_review_count][2] = review_count[cur_review_count][2] + 1
df_review_count = pd.DataFrame.from_dict(review_count).T
df_review_count.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_review_count['review_count'] = range(100)
df_review_count.to_csv('./graph_review.csv', index=False)  

In [122]:
df_review_count_percentage = pd.DataFrame()
df_review_count_percentage['review_count'] = df_review_count['review_count']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_review_count.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_review_count_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_review_count_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_review_count_percentage['OPERATIONAL'] = OPERATIONAL
df_review_count_percentage.to_csv('./graph_review_count_percentage.csv', index=False)  

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in longlong_scalars

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in longlong_scalars

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning:

invalid value encountered in longlong_scalars



checkIn_2019

In [154]:
df_raw3 = pd.read_csv("./没有sample的最终.csv")
df_checkIn_2019 = df_raw3[['checkIn_2019', 'status_g']]
df_checkIn_2019 = df_checkIn_2019.dropna()
df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'None']
df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'FALSE']
df_checkIn_2019 = df_checkIn_2019.astype({"checkIn_2019": float})
df_checkIn_2019 = df_checkIn_2019.astype({"checkIn_2019": int})
df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 <= 100]
len(df_checkIn_2019[df_checkIn_2019.checkIn_2019 > 50])

2777

In [150]:
checkIn_2019 = {}
for item in range(100):
    checkIn_2019[item] = [0, 0, 0]
for index, row in df_checkIn_2019.iterrows():
    cur_checkIn_2019 = row['checkIn_2019']
    if cur_checkIn_2019 not in checkIn_2019.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        checkIn_2019[cur_checkIn_2019][0] = checkIn_2019[cur_checkIn_2019][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        checkIn_2019[cur_checkIn_2019][1] = checkIn_2019[cur_checkIn_2019][1] + 1
    elif cur_status == 'OPERATIONAL':
        checkIn_2019[cur_checkIn_2019][2] = checkIn_2019[cur_checkIn_2019][2] + 1
df_checkIn_2019 = pd.DataFrame.from_dict(checkIn_2019).T
df_checkIn_2019.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_checkIn_2019['checkIn_2019'] = range(100)
df_checkIn_2019.to_csv('./graph_checkIn_2019.csv', index=False)  

In [151]:
df_checkIn_2019_percentage = pd.DataFrame()
df_checkIn_2019_percentage['checkIn_2019'] = df_checkIn_2019['checkIn_2019']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_checkIn_2019.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_checkIn_2019_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_checkIn_2019_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_checkIn_2019_percentage['OPERATIONAL'] = OPERATIONAL
df_checkIn_2019_percentage.to_csv('./graph_checkIn_2019_percentage.csv', index=False)  

total_hours_week

In [138]:
df_clean = pd.read_csv("./没有sample的最终.csv")
df_clean = df_clean.fillna('None')
if 'hours' in df_clean.columns:
    total_hours = []
    for index, row in df_clean.iterrows():
            current = row['hours']
            #print(type(current))
            if current == 0 or current == 'None' or 'casual' in current:
                total_hours.append(0)
                continue
            summ = 0
            for key in (ast.literal_eval(current)).keys():
                cur_duration = (ast.literal_eval(current))[key]
                index = cur_duration.find('-')
                before = cur_duration[:index]
                after = cur_duration[index+1:]
                before = datetime.strptime(before, "%H:%M")
                after = datetime.strptime(after, "%H:%M")
                #print(cur_duration)
                #print(abs(divmod((after-before).total_seconds(), 3600)[0]))
                summ = summ + abs(divmod((after-before).total_seconds(), 3600)[0])
            total_hours.append(summ)
    df_clean['total_hours_week'] = total_hours

In [159]:
df_total_hours_week = df_clean[['total_hours_week', 'status_g']]
df_total_hours_week = df_total_hours_week.dropna()
df_total_hours_week = df_total_hours_week.astype({"total_hours_week": float})
df_total_hours_week = df_total_hours_week.astype({"total_hours_week": int})
df_total_hours_week = df_total_hours_week[df_total_hours_week.total_hours_week <= 110]
len(df_total_hours_week[df_total_hours_week.total_hours_week > 110])

0

In [160]:
total_hours_week = {}
for item in range(110):
    total_hours_week[item] = [0, 0, 0]
for index, row in df_total_hours_week.iterrows():
    cur_total_hours_week = row['total_hours_week']
    if cur_total_hours_week not in total_hours_week.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        total_hours_week[cur_total_hours_week][0] = total_hours_week[cur_total_hours_week][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        total_hours_week[cur_total_hours_week][1] = total_hours_week[cur_total_hours_week][1] + 1
    elif cur_status == 'OPERATIONAL':
        total_hours_week[cur_total_hours_week][2] = total_hours_week[cur_total_hours_week][2] + 1
df_total_hours_week = pd.DataFrame.from_dict(total_hours_week).T
df_total_hours_week.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_total_hours_week['total_hours_week'] = range(110)
df_total_hours_week.to_csv('./graph_total_hours_week.csv', index=False)  

In [161]:
df_total_hours_week_percentage = pd.DataFrame()
df_total_hours_week_percentage['total_hours_week'] = df_total_hours_week['total_hours_week']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_total_hours_week.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_total_hours_week_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_total_hours_week_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_total_hours_week_percentage['OPERATIONAL'] = OPERATIONAL
df_total_hours_week_percentage.to_csv('./graph_total_hours_week_percentage.csv', index=False)  

total_checkIn

In [169]:
df_raw4 = pd.read_csv("./没有sample的最终.csv")
df_total_checkIn = df_raw4[['total_checkIn', 'status_g']]
df_total_checkIn = df_total_checkIn.dropna()
df_total_checkIn = df_total_checkIn[df_total_checkIn.total_checkIn != 'None']
df_total_checkIn = df_total_checkIn[df_total_checkIn.total_checkIn != 'FALSE']
df_total_checkIn = df_total_checkIn.astype({"total_checkIn": float})
df_total_checkIn = df_total_checkIn.astype({"total_checkIn": int})
df_total_checkIn = df_total_checkIn[df_total_checkIn.total_checkIn <= 1000]
#len(df_total_checkIn[df_total_checkIn.total_checkIn > 1000])
df_total_checkIn.total_checkIn.describe()

count    52183.000000
mean       112.891478
std        177.250499
min          1.000000
25%         10.000000
50%         38.000000
75%        127.000000
max       1000.000000
Name: total_checkIn, dtype: float64

In [170]:
total_checkIn = {}
for item in range(1000):
    total_checkIn[item] = [0, 0, 0]
for index, row in df_total_checkIn.iterrows():
    cur_total_checkIn = row['total_checkIn']
    if cur_total_checkIn not in total_checkIn.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        total_checkIn[cur_total_checkIn][0] = total_checkIn[cur_total_checkIn][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        total_checkIn[cur_total_checkIn][1] = total_checkIn[cur_total_checkIn][1] + 1
    elif cur_status == 'OPERATIONAL':
        total_checkIn[cur_total_checkIn][2] = total_checkIn[cur_total_checkIn][2] + 1
df_total_checkIn = pd.DataFrame.from_dict(total_checkIn).T
df_total_checkIn.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_total_checkIn['total_checkIn'] = range(1000)
df_total_checkIn.to_csv('./graph_total_checkIn.csv', index=False)  

In [171]:
df_total_checkIn_percentage = pd.DataFrame()
df_total_checkIn_percentage['total_checkIn'] = df_total_checkIn['total_checkIn']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_total_checkIn.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_total_checkIn_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_total_checkIn_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_total_checkIn_percentage['OPERATIONAL'] = OPERATIONAL
df_total_checkIn_percentage.to_csv('./graph_total_checkIn_percentage.csv', index=False)  

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in longlong_scalars

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in longlong_scalars

c:\users\xinyi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning:

invalid value encountered in longlong_scalars



dayOfStart

In [190]:
df_raw5 = pd.read_csv("./没有sample的最终.csv")
df_dayOfStart = df_raw4[['dayOfStart', 'status_g']]
df_dayOfStart = df_dayOfStart.dropna()
df_dayOfStart = df_dayOfStart[df_dayOfStart.dayOfStart != 'None']
df_dayOfStart = df_dayOfStart[df_dayOfStart.dayOfStart != 'FALSE']
df_dayOfStart = df_dayOfStart.astype({"dayOfStart": float})
df_dayOfStart = df_dayOfStart.astype({"dayOfStart": int})
df_dayOfStart = df_dayOfStart[df_dayOfStart.dayOfStart <= 3800]
#len(df_dayOfStart[df_dayOfStart.dayOfStart > 3800])
df_dayOfStart.dayOfStart.describe()

count    54719.000000
mean      2419.522981
std       1103.148616
min        179.000000
25%       1467.000000
50%       2692.000000
75%       3456.000000
max       3798.000000
Name: dayOfStart, dtype: float64

In [191]:
dayOfStart = {}
for item in range(3801):
    dayOfStart[item] = [0, 0, 0]
for index, row in df_dayOfStart.iterrows():
    cur_dayOfStart = row['dayOfStart']
    if cur_dayOfStart not in dayOfStart.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        dayOfStart[cur_dayOfStart][0] = dayOfStart[cur_dayOfStart][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        dayOfStart[cur_dayOfStart][1] = dayOfStart[cur_dayOfStart][1] + 1
    elif cur_status == 'OPERATIONAL':
        dayOfStart[cur_dayOfStart][2] = dayOfStart[cur_dayOfStart][2] + 1
df_dayOfStart = pd.DataFrame.from_dict(dayOfStart).T
df_dayOfStart.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
df_dayOfStart['dayOfStart'] = range(3801)
df_dayOfStart.to_csv('./graph_dayOfStart.csv', index=False)  

In [192]:
df_dayOfStart = df_dayOfStart.drop(df_dayOfStart[(df_dayOfStart['CLOSED_PERMANENTLY'] == 0) & (df_dayOfStart['CLOSED_TEMPORARILY'] == 0) & (df_dayOfStart['OPERATIONAL'] == 0)].index)

In [193]:
df_dayOfStart_percentage = pd.DataFrame()
df_dayOfStart_percentage['dayOfStart'] = df_dayOfStart['dayOfStart']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_dayOfStart.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_dayOfStart_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_dayOfStart_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_dayOfStart_percentage['OPERATIONAL'] = OPERATIONAL
df_dayOfStart_percentage.to_csv('./graph_dayOfStart_percentage.csv', index=False)  

death

In [86]:
df_raw4 = pd.read_csv("./没有sample的最终.csv")

In [78]:
df_raw4

,Unnamed: 0,business_id,Cases,Deaths,name,address,city,state,postal_code,latitude,...,categories_Asian Fusion,categories_Mediterranean,total_checkIn,checkIn_2019,chain,covid_cumul_case_state,covid_cumul_death_state,dayOfStart,land_price,status_g
0,0,eBEfgOPG7pvFhb2wcG9I7w,2650.0,222.0,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,...,False,False,1.0,1,True,32033.0,2484.0,522,11225,OPERATIONAL
1,1,lu7vtrp_bE9PnxWfA8g4Pg,2650.0,222.0,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,...,False,False,8.0,0,False,32033.0,2484.0,2020,11225,OPERATIONAL
2,2,9sRGfSVEfLhN_km60YruTA,2650.0,222.0,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,...,False,False,2.0,2,False,32033.0,2484.0,443,11225,CLOSED_TEMPORARILY
3,3,pcaQDBM6r0PWTXfYZK6RdA,26288.0,3068.0,IGA Extra,8130 Boulevard Champlain,Lasalle,QC,H8P 1B4,45.425645,...,False,False,181.0,13,False,52849.0,4978.0,3266,6606,OPERATIONAL
4,4,LoRef3ChgZKbxUio-sHgQg,26288.0,3068.0,Amir,5252 Rue Jean Talon O,MontrÂ©al,QC,H4P 2A7,45.494870,...,False,False,10.0,0,True,52849.0,4978.0,2575,6606,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56169,56169,AlwKuzyrrxlZ04ktebwmcA,824.0,29.0,Pizza Pit,13 Atlas Ct,Madison,WI,53714,43.086793,...,False,False,7.0,0,True,20701.0,646.0,3542,70070,OPERATIONAL
56170,56170,cmsAFAymfVoBitSF8R6GKw,12761.0,489.0,Yao Fine Chinese Cuisine,"2487 S Gilbert Rd, Ste 102",Gilbert,AZ,85295,33.304935,...,False,False,419.0,23,False,25517.0,1047.0,3766,26493,OPERATIONAL
56171,56171,YZeUH6zYS0dq5QHLYZhUnQ,12761.0,489.0,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,...,False,False,253.0,9,True,25517.0,1047.0,3076,26493,OPERATIONAL
56172,56172,hskVqZCPqy-omm9CHi44xQ,5358.0,106.0,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,...,False,False,6.0,0,True,34809.0,1020.0,3396,38976,OPERATIONAL


In [87]:
df_raw4 = pd.read_csv("./没有sample的最终.csv")
df_Deaths = df_raw4[['Deaths', 'status_g']]
df_Deaths = df_Deaths.dropna()
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'None']
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'FALSE']
df_Deaths = df_Deaths.astype({"Deaths": float})
df_Deaths = df_Deaths.astype({"Deaths": int})
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 <= 50]
df_Deaths['Deaths'].describe()

count    56174.000000
mean       589.797130
std        768.873081
min          0.000000
25%        168.000000
50%        358.000000
75%        489.000000
max       3913.000000
Name: Deaths, dtype: float64

In [88]:
Deaths = {}
for item in range(3913):
    Deaths[item] = [0, 0, 0]
for index, row in df_Deaths.iterrows():
    cur_Deaths = row['Deaths']
    if cur_Deaths not in Deaths.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        Deaths[cur_Deaths][0] = Deaths[cur_Deaths][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        Deaths[cur_Deaths][1] = Deaths[cur_Deaths][1] + 1
    elif cur_status == 'OPERATIONAL':
        Deaths[cur_Deaths][2] = Deaths[cur_Deaths][2] + 1
df_Deaths = pd.DataFrame.from_dict(Deaths).T
df_Deaths.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
#df_Deaths = df_Deaths[]
df_Deaths['Deaths'] = range(3913)
# df_Deaths.to_csv('./graph_Deaths.csv', index=False)  

In [92]:
df_Deaths = df_Deaths.drop(df_Deaths[(df_Deaths['CLOSED_PERMANENTLY'] == 0) & (df_Deaths['CLOSED_TEMPORARILY'] == 0) & (df_Deaths['OPERATIONAL'] == 0)].index)

In [98]:
df_Deaths_percentage = pd.DataFrame()

In [99]:
df_Deaths_percentage['Deaths'] = df_Deaths['Deaths']

In [104]:
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_Deaths.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)


In [105]:
df_Deaths_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_Deaths_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_Deaths_percentage['OPERATIONAL'] = OPERATIONAL

In [106]:
df_Deaths_percentage.to_csv('./graph_Deaths_percentage.csv', index=False)  

In [93]:
df_Deaths.to_csv('./graph_Deaths.csv', index=False)  

Case

In [194]:
df_raw6 = pd.read_csv("./没有sample的最终.csv")
df_Cases = df_raw6[['Cases', 'status_g']]
df_Cases = df_Cases.dropna()
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'None']
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 != 'FALSE']
df_Cases = df_Cases.astype({"Cases": float})
df_Cases = df_Cases.astype({"Cases": int})
#df_checkIn_2019 = df_checkIn_2019[df_checkIn_2019.checkIn_2019 <= 50]
df_Cases['Cases'].describe()

count    56174.000000
mean      8755.855841
std       6705.818040
min          7.000000
25%       4827.000000
50%       7331.000000
75%      12761.000000
max      81924.000000
Name: Cases, dtype: float64

In [195]:
Cases = {}
for item in range(81925):
    Cases[item] = [0, 0, 0]
for index, row in df_Cases.iterrows():
    cur_Cases = row['Cases']
    if cur_Cases not in Cases.keys():
        continue
    cur_status = row['status_g']
    if cur_status == 'CLOSED_PERMANENTLY':
        Cases[cur_Cases][0] = Cases[cur_Cases][0] + 1
    elif cur_status == 'CLOSED_TEMPORARILY':
        Cases[cur_Cases][1] = Cases[cur_Cases][1] + 1
    elif cur_status == 'OPERATIONAL':
        Cases[cur_Cases][2] = Cases[cur_Cases][2] + 1
df_Cases = pd.DataFrame.from_dict(Cases).T
df_Cases.columns = ['CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', 'OPERATIONAL']
#df_Deaths = df_Deaths[]
df_Cases['Cases'] = range(81925)
# df_Deaths.to_csv('./graph_Deaths.csv', index=False)  

In [196]:
df_Cases = df_Cases.drop(df_Cases[(df_Cases['CLOSED_PERMANENTLY'] == 0) & (df_Cases['CLOSED_TEMPORARILY'] == 0) & (df_Cases['OPERATIONAL'] == 0)].index)

In [197]:
df_Cases_percentage = pd.DataFrame()
df_Cases_percentage['Cases'] = df_Cases['Cases']
CLOSED_PERMANENTLY = []
CLOSED_TEMPORARILY = []
OPERATIONAL = []
for index, row in df_Cases.iterrows():
    cur_CLOSED_PERMANENTLY = row['CLOSED_PERMANENTLY']
    cur_CLOSED_TEMPORARILY = row['CLOSED_TEMPORARILY']
    cur_OPERATIONAL = row['OPERATIONAL']
    cur_sum = cur_CLOSED_PERMANENTLY + cur_CLOSED_TEMPORARILY + cur_OPERATIONAL
    CLOSED_PERMANENTLY.append(cur_CLOSED_PERMANENTLY/cur_sum*100)
    CLOSED_TEMPORARILY.append(cur_CLOSED_TEMPORARILY/cur_sum*100)
    OPERATIONAL.append(cur_OPERATIONAL/cur_sum*100)
df_Cases_percentage['CLOSED_PERMANENTLY'] = CLOSED_PERMANENTLY
df_Cases_percentage['CLOSED_TEMPORARILY'] = CLOSED_TEMPORARILY
df_Cases_percentage['OPERATIONAL'] = OPERATIONAL
df_Cases_percentage.to_csv('./graph_Cases_percentage.csv', index=False)  

In [198]:
df_raw6 = pd.read_csv("./没有sample的最终.csv")

In [199]:
df_raw6

,Unnamed: 0,business_id,Cases,Deaths,name,address,city,state,postal_code,latitude,...,categories_Asian Fusion,categories_Mediterranean,total_checkIn,checkIn_2019,chain,covid_cumul_case_state,covid_cumul_death_state,dayOfStart,land_price,status_g
0,0,eBEfgOPG7pvFhb2wcG9I7w,2650.0,222.0,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,...,False,False,1.0,1,True,32033.0,2484.0,522,11225,OPERATIONAL
1,1,lu7vtrp_bE9PnxWfA8g4Pg,2650.0,222.0,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,...,False,False,8.0,0,False,32033.0,2484.0,2020,11225,OPERATIONAL
2,2,9sRGfSVEfLhN_km60YruTA,2650.0,222.0,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,...,False,False,2.0,2,False,32033.0,2484.0,443,11225,CLOSED_TEMPORARILY
3,3,pcaQDBM6r0PWTXfYZK6RdA,26288.0,3068.0,IGA Extra,8130 Boulevard Champlain,Lasalle,QC,H8P 1B4,45.425645,...,False,False,181.0,13,False,52849.0,4978.0,3266,6606,OPERATIONAL
4,4,LoRef3ChgZKbxUio-sHgQg,26288.0,3068.0,Amir,5252 Rue Jean Talon O,MontrÂ©al,QC,H4P 2A7,45.494870,...,False,False,10.0,0,True,52849.0,4978.0,2575,6606,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56169,56169,AlwKuzyrrxlZ04ktebwmcA,824.0,29.0,Pizza Pit,13 Atlas Ct,Madison,WI,53714,43.086793,...,False,False,7.0,0,True,20701.0,646.0,3542,70070,OPERATIONAL
56170,56170,cmsAFAymfVoBitSF8R6GKw,12761.0,489.0,Yao Fine Chinese Cuisine,"2487 S Gilbert Rd, Ste 102",Gilbert,AZ,85295,33.304935,...,False,False,419.0,23,False,25517.0,1047.0,3766,26493,OPERATIONAL
56171,56171,YZeUH6zYS0dq5QHLYZhUnQ,12761.0,489.0,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,...,False,False,253.0,9,True,25517.0,1047.0,3076,26493,OPERATIONAL
56172,56172,hskVqZCPqy-omm9CHi44xQ,5358.0,106.0,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,...,False,False,6.0,0,True,34809.0,1020.0,3396,38976,OPERATIONAL
